# Pulse duration - data analysis

Analysis of the pulse duration data.

In [1]:
%matplotlib widget
from core.experiment import *

# Load data

In [2]:
root_path = "D:\\Data_LIBS\\Miguel\\Exp Pulse\\Target Fe 99,95% LOT005133\\"

folders = [f for f in os.listdir(root_path)]

list_of_experiments=[]
heating_time = []

for i in range(0,len(folders)):
    folder = folders[i]
    sample_folder = root_path + folder +"\\"
    list_of_experiments.append(experiment(sample_folder))#,ignore='0000'
    try:
        heating_time.append(int(folder.split(' ')[0]))
    except:
        heating_time.append(folder)


# Compute the temperature

In [3]:
root_path = "D:\\Data_LIBS\\Miguel\\Exp Pulse\\Target Fe 99,95% LOT005133\\"

folders = [f for f in os.listdir(root_path)]

list_of_experiments=[]
heating_time = []

for i in range(0,len(folders)):
    folder = folders[i]
    sample_folder = root_path + folder +"\\"
    list_of_experiments.append(experiment(sample_folder))#,ignore='0000'
    try:
        heating_time.append(int(folder.split(' ')[0]))
    except:
        heating_time.append(folder)


#Choose the element for computing the relevant lines
al = element("Fe")

#reference temperature
Tp = 1.*T_ref
density = n_e_ref

##signal
temperatures=[]
temps_95=[]
for i in range(0,len(list_of_experiments)-1):
    
    print(heating_time[i])
    
    sg = list_of_experiments[i].mean_signal
    back=list_of_experiments[-1].mean_signal
    
    sg.spectrum[0] = sg.spectrum[0] - back.spectrum[0]  
    sg.remove_baseline()

    #define the upper and lower limits to find the most relevant nlines
    #let us define to the
    n_spectrometer = 0
    ll = sg.wavelengths[n_spectrometer][0]#300
    #ll = 400
    ul = sg.wavelengths[n_spectrometer][-1]
    ul= 390

    #ion state 1
    nlines = 10
    l_num, lines = al.get_most_relevant_lines_ion_state(ion_state = 1, electron_temperature=Tp, n_lines = nlines,lower_limit = ll, upper_limit = ul)
    
    lines=np.delete(lines,[6,1])
    
    
    #ion state 2
    nlines = 2
    l_num2, lines2 = al.get_most_relevant_lines_ion_state(ion_state = 2, electron_temperature=Tp, n_lines = nlines,lower_limit = ll, upper_limit = ul)
    ion_energies = al.ion_energies
    lines2=np.delete(lines2,[0])

    #compute the plasma temperature
    temperature, temp_95, r2, y_s, x_s = sg.saha_boltzmann_temperature_v2([lines],al,electron_density= density, 
                                                                          ratio_of_maximum = 0.5, radius = 0.5, Plot = True, Plotlines = True, use_max_intensity = False)

    temperatures.append(temperature)
    temps_95.append(temp_95)

10


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

100


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

75


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Compare with a synthetic spectrum

In [4]:
sample_elements = [['Fe',1]]
sample = digital_twin(sample_elements)

print(sample)

temperature = 1.*T_ref
sg = list_of_experiments[0].mean_signal
sg.remove_baseline()

for n_spectrometer in range(0,len(sg.wavelengths)):
    
    
    ll = sg.wavelengths[n_spectrometer][0]
    ul = sg.wavelengths[n_spectrometer][-1]

    #ion state 1
    nlines = 10
    l_num, lines = al.get_most_relevant_lines_ion_state(ion_state = 1, electron_temperature = temperature, 
                                                        n_lines = nlines,lower_limit = ll, upper_limit = ul)

    #line to normalize the intensity for direct
    line_norm1=lines[5]
    print("# Spectrometer 1 normalized to " + str(line_norm1))

    sg.compare_to_digital_sample(sample, spectrometer=n_spectrometer, 
                                 electron_temperature = temperature,electron_density=density,
                                 d_lambda=0.02, use_wavelengths=True,line_normalize=None,
                                 resolution = 2000, Plotline = True)

Digital sample with 
 [ Fe ratio 1 ] 
# Spectrometer 1 normalized to Line of Wavelength 404.58119300000004



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Variation of the intensity of a line

In [9]:
durations = []
total_int=[]
nions=[]
for j in range(0,len(list_of_experiments)-1):
    print(j)
    current_experiment = list_of_experiments[j]

    intensities=[]

    for i in range(0, len(current_experiment.list_of_signals)):
        #subplots()
        ratio_of_maximum = 0.1
        wavelengths = np.concatenate([w for w in current_experiment.list_of_signals[i].wavelengths]) 
        spectrum = np.concatenate([s for s in current_experiment.list_of_signals[i].spectrum ])
        radius = .3
        ritz = 404.58#578.21
        intensity = get_peak_area(ritz,ratio_of_maximum , wavelengths, spectrum, radius, False)
        intensities.append(intensity)
    
    temperature = temperatures[j]
    wl,spec,label, n_ion, specs = sample.spectrum_NIST( lower_limit = ll, upper_limit = ul, electron_temperature=temperature, 
                                                   max_ion_state=3,d_lambda=0.1, Map=False)
    
    Z_ion = partition_function(al.label,str(0),temperature/T_ref)
    
    nions.append(n_ion/Z_ion)
    intensities=array(intensities)
    durations.append(heating_time[j]*ones(intensities.shape))
    total_int.append(intensities)
    

    
nions=np.array(nions)    
durations=np.array(durations)
total_int = np.array(total_int)
std_d = np.std(total_int,axis=1)

#aaa = array([array([yy,xx]) for yy,xx in sorted(zip(times,areas_mean-areas_error_bars[0,:]))])
mean_durations = [y for y, x in sorted(zip(np.mean(durations,axis=1),np.mean(total_int,axis=1)), key=lambda pair: pair[0])]
mean_intensities = [x for y, x in sorted(zip(np.mean(durations,axis=1),np.mean(total_int,axis=1)), key=lambda pair: pair[0])]
std_d_ord = [x for y, x in sorted(zip(np.mean(durations,axis=1),std_d), key=lambda pair: pair[0])]
mean_temperatures = [x for y, x in sorted(zip(np.mean(durations,axis=1),temperatures), key=lambda pair: pair[0])]
mean_nions = [x for y, x in sorted(zip(np.mean(durations,axis=1),nions[:,0,0]), key=lambda pair: pair[0])]


fig,ax = subplots(figsize=[6,5])
subplot(211)
ax1 = gca()
plot(durations,total_int/mean_intensities[0],'.',color = "gray")
#plot(mean(durations,axis=1),mean(total_int,axis=1),'-',lw=0.5,marker = 'o',color='k')
errorbar(mean_durations,mean_intensities/mean_intensities[0],yerr=std_d/mean_intensities[0],color='k',lw=1,elinewidth=1,capsize=5)
xlabel(r"Pulse duration $\tau_p$ (ns)")
ylabel(r"$\frac{I}{I_{10ns}}$ at 578.2nm (arb.un.)")

subplot(212)

ax2= gca()

ax2.set_ylabel(r'$T_{plasma}$ (eV)')  # we already handled the x-label with ax1
ax2.plot(mean_durations, np.array(mean_temperatures)/T_ref, '--',marker = 'o', color='k')

ax1 = ax2.twinx()  # instantiate a second axes that shares the same x-axis

ax1.plot(mean_durations,mean_nions, ':',marker = 'x', color='gray')
ax1.set_ylabel(r'Predicted $R(T_{p})$ (arb.un.)')
ax2.set_xlabel(r"Pulse duration $\tau_p$ (ns)")
fig.tight_layout()  # otherwise the right y-label is slightly clipped

0


C:\Users\nunoa\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:27: DeprecationWarning: scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead
C:\Users\nunoa\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:28: DeprecationWarning: scipy.ones is deprecated and will be removed in SciPy 2.0.0, use numpy.ones instead


1
2
3
4


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
d = [x for _, x in sorted(zip(mean(durations,axis=1),mean(total_int,axis=1)), key=lambda pair: pair[0])]
c = [y for y, x in sorted(zip(mean(durations,axis=1),mean(total_int,axis=1)), key=lambda pair: pair[0])]
c

# Computing density
Using the H_alpha line from the Balmer series.

In [ ]:
fig,ax=subplots(figsize=[8,3])
sg=list_of_experiments[0].mean_signal
plot(sg.wavelengths[0], sg.spectrum[0], '-',  label = r"$\tau_p = $" + str(heating_time[0]) + " $ns$")

#sg=list_of_experiments[2].mean_signal
#plot(sg.wavelengths[0], sg.spectrum[0], ':', label = r"$\tau_p = $" + str(heating_time[2]) + " $ns$")
sg=list_of_experiments[3].mean_signal
plot(sg.wavelengths[0], sg.spectrum[0], '-.', label = r"$\tau_p = $" + str(heating_time[3]) + " $ns$")
#sg=list_of_experiments[4].mean_signal
#plot(sg.wavelengths[0], sg.spectrum[0], '-.', label = r"$\tau_p = $" + str(heating_time[4]) + " $ns$")

sg=list_of_experiments[1].mean_signal
plot(sg.wavelengths[0], sg.spectrum[0], '--', label = r"$\tau_p = $" + str(heating_time[1]) + " $ns$")
xlabel("Wavelength (nm)")
ylabel("Intensity (arb.un.)")
xlim(508,524)
#for i in range(0,len(list_of_experiments)-1):
#    sg=list_of_experiments[i].mean_signal
    #back=list_of_experiments[-1].mean_signal
#    plot(sg.wavelengths[0], sg.spectrum[0], '-', label = str(heating_time[i]))
legend(fontsize=12)
fig.tight_layout()

In [ ]:
fig,ax=subplots(figsize=[8,3])

pulsesx = []
pulsesy = []

durat = [10,20,50,75,100]
colsx = ["s 10","s 20","s 50","s 75","s 100"]
colsy = ["I 10","I 20","I 50","I 75","I 100"]

df = pd.read_csv("D:\\Data_LIBS\\pulse.csv",sep=";")

for i in range(0,len(colsx)):
    pulsesx.append(df[colsx[i]].to_numpy())
    pulsesy.append(df[colsy[i]].to_numpy())
    plot(pulsesx[-1]*10e8+2,pulsesy[-1],label=r"$\tau_p = $"+str(durat[i])+ ' ns')

xlabel("Time (ns)")
ylabel("Pulse power (arb.un.)")
xlim(0,150)
#for i in range(0,len(list_of_experiments)-1):
#    sg=list_of_experiments[i].mean_signal
    #back=list_of_experiments[-1].mean_signal
#    plot(sg.wavelengths[0], sg.spectrum[0], '-', label = str(heating_time[i]))
legend(fontsize=12)
fig.tight_layout()

# Computing the plasma temperature
Using the Saha-Boltzmann method and choosing the lines

Using Na

In [ ]:
#Choose the element for computing the relevant lines
al = element('Na')

Tp = 1.*T_ref

#define the upper and lower limits to find the most relevant nlines
#let us define to the
n_spectrometer = 5
ll = sg.wavelengths[n_spectrometer][0]
ul = sg.wavelengths[n_spectrometer][-1]

#ion state 1
nlines = 5
l_num, lines = al.get_most_relevant_lines_ion_state(ion_state = 1, electron_temperature=Tp, n_lines = nlines,lower_limit = ll, upper_limit = ul)

#ion state 2
nlines = 2
l_num2, lines2 = al.get_most_relevant_lines_ion_state(ion_state = 2, electron_temperature=Tp, n_lines = nlines,lower_limit = ll, upper_limit = ul)

#ion state 3
nlines = 2
l_num3, lines3 = al.get_most_relevant_lines_ion_state(ion_state = 3, electron_temperature=Tp, n_lines = nlines,lower_limit = ll, upper_limit = ul)
ion_energies = al.ion_energies

#compute the plasma temperature
temperature, temp_95, r2, y_s, x_s = sg.saha_boltzmann_temperature_v2([lines,lines2],ion_energies,electron_density= density, 
                                                                      ratio_of_maximum = 0.5, radius = 0.1, Plot = True, Plotlines = True, use_max_intensity = False)


Using Cl

In [ ]:
#Choose the element for computing the relevant lines
al = element('Cl')

Tp = 1.*T_ref

#define the upper and lower limits to find the most relevant nlines
#let us define to the
n_spectrometer = 4
ll = sg.wavelengths[n_spectrometer][0]
ul = sg.wavelengths[n_spectrometer][-1]

#ion state 1
nlines = 5
l_num, lines = al.get_most_relevant_lines_ion_state(ion_state = 1, electron_temperature=Tp, n_lines = nlines,lower_limit = ll, upper_limit = ul)

#ion state 2
nlines = 2
l_num2, lines2 = al.get_most_relevant_lines_ion_state(ion_state = 2, electron_temperature=Tp, n_lines = nlines,lower_limit = ll, upper_limit = ul)

#ion state 3
nlines = 2
l_num3, lines3 = al.get_most_relevant_lines_ion_state(ion_state = 3, electron_temperature=Tp, n_lines = nlines,lower_limit = ll, upper_limit = ul)
ion_energies = al.ion_energies

#compute the plasma temperature
temperature, temp_95, r2, y_s, x_s = sg.saha_boltzmann_temperature_v2([lines,lines2],ion_energies,electron_density= density, 
                                                                      ratio_of_maximum = 0.5, radius = 0.1, Plot = True, Plotlines = True, use_max_intensity = False)


# LTE conditions
Probing the Mcwrither criterium

In [ ]:
minimum_density = sg.LTE_MW_criterium(4.8,temperature)
print("Minimum density in cm^-3: " + str("%e" %minimum_density))
print("Computed density in cm^-3: " + str("%e" %(density*(10**-2)**3)))
print("MW criterion - " + str(density>(minimum_density*(10**2)**3)))

In [ ]:
al = element('Na')
sample_elements = [['Na',.5],['Cl',.5]]
sample = digital_twin(sample_elements)

print(sample)

for n_spectrometer in range(0,len(sg.wavelengths)):
    ll = sg.wavelengths[n_spectrometer][0]
    ul = sg.wavelengths[n_spectrometer][-1]

    #ion state 1
    nlines = 3
    l_num, lines = al.get_most_relevant_lines_ion_state(ion_state = 1, electron_temperature=Tp, 
                                                        n_lines = nlines,lower_limit = ll, upper_limit = ul)

    #line to normalize the intensity for direct
    line_norm1=lines[1]
    print("# Spectrometer 1 normalized to " + str(line_norm1))

    sg.compare_to_digital_sample(sample, spectrometer=n_spectrometer, 
                                 electron_temperature = temperature,electron_density=density,
                                 d_lambda=0.02, use_wavelengths=True,line_normalize=line_norm1,Plotline = True)

In [ ]:
a=array([1,2])
b=copy(a)
b*=2
a

In [ ]:



ll = 200.
ul = 900.
Tp = 1.*T_ref
new_wavelengths = np.concatenate([wl for wl in sg.wavelengths])
#wl,spec,label, n_ion = al.synthetic_spectrum_NIST(wl = new_wavelengths, lower_limit = ll, upper_limit = ul, electron_temperature=Tp,max_ion_state=3,d_lambda=0.1)
wl,spec,label, n_ion, specs = sample.spectrum_NIST(wl = new_wavelengths, lower_limit = ll, upper_limit = ul, electron_temperature=Tp,max_ion_state=3,d_lambda=0.01, Map=True)
wl=wl[0]



In [ ]:
#define the upper and lower limits to find the most relevant nlines
#let us define to the
x_ss=[]
y_ss=[]
for n_spectrometer in range(0,6):
    ll = sg.wavelengths[n_spectrometer][0]
    ul = sg.wavelengths[n_spectrometer][-1]

    #ion state 1
    nlines = 5
    l_num, lines = al.get_most_relevant_lines_ion_state(ion_state = 1, electron_temperature=Tp, n_lines = nlines,lower_limit = ll, upper_limit = ul)

    #ion state 2
    nlines = 2
    l_num2, lines2 = al.get_most_relevant_lines_ion_state(ion_state = 2, electron_temperature=Tp, n_lines = nlines,lower_limit = ll, upper_limit = ul)

    #ion state 3
    nlines = 2
    l_num3, lines3 = al.get_most_relevant_lines_ion_state(ion_state = 3, electron_temperature=Tp, n_lines = nlines,lower_limit = ll, upper_limit = ul)
    ion_energies = al.ion_energies

    #compute the plasma temperature
    temperature1, temp_95, r2, y_s, x_s = sg.saha_boltzmann_temperature_v2([lines,lines2],ion_energies,electron_density= density, 
                                                                          ratio_of_maximum = 0.5, radius = 0.1, Plot = False, Plotlines = False, use_max_intensity = False)

    x_ss.append(x_s)
    y_ss.append(y_s)

In [ ]:
def density_copper(signal0,fit="Voigt",Plot=True, compare_methods = False,ratio_max=0.1):
            
        
        line_ritz = 515.3
        new_wavelengths = np.concatenate([wl for wl in signal0.wavelengths])
        new_spectrum = np.concatenate([sp for sp in signal0.spectrum])
        new_spectrum += -baseline_als(new_spectrum,100000,0.05)
        center_wavelength_peak_index = get_closest_peak_index(line_ritz, ratio_max, new_wavelengths, new_spectrum)
        index_lr = get_closest_peak_width_index(center_wavelength_peak_index,ratio_max, new_wavelengths, new_spectrum)
        
        if compare_methods==False:
            if fit == "Voigt":
                current_profile_binder = voigt_profile_binder
            elif fit == "Gaussian":
                current_profile_binder = gaussian_profile_binder
            else:
                current_profile_binder = lorentzian_profile_binder
            
            params, pcov = binder_fit(new_wavelengths[index_lr[0]:index_lr[1]], new_spectrum[index_lr[0]:index_lr[1]],
                                     current_profile_binder,
                                     initial_guess_x0 = line_ritz, initial_guess_A = new_spectrum[center_wavelength_peak_index])
            
            xx = np.arange(new_wavelengths[index_lr[0]],new_wavelengths[index_lr[1]],0.001)
            fwhm, fwhm_x_v, fwhm_y_v = fit_fwhm(xx,params,current_profile_binder)
            
            #density = 10e17*pow(fwhm/1.31,1/0.64)
            #density = 9.77e16*pow(fwhm,1.39)
            ww = 0.0102
            density = (fwhm/(2*ww))*10**16
            if Plot:
                figure()
                title(r'$Copper_{(510.5 nm)}$ line fit',fontsize=12)
                plot(new_wavelengths[index_lr[0]:index_lr[1]],new_spectrum[index_lr[0]:index_lr[1]],'-',color='k',label='Signal')
                fill_between(xx,current_profile_binder(xx,params[0],params[1],params[2],params[3]),color='b',alpha=0.2,label=fit+' fit')
                plot(fwhm_x_v,fwhm_y_v,ls = ':', color = 'k', marker = '|',markersize = 10)
                ax = gca()
                ax.text(0.3, 0.2,  '$n_{plasma} =$' + "%1.2e" % density +' $cm^{-3}$',
                transform=ax.transAxes,fontsize=12, bbox={'boxstyle':'round', 'facecolor': 'wheat', 'alpha': 0.5, 'pad': 0.5})
                ax.xaxis.set_major_formatter(ticker.FormatStrFormatter('%0.1f'))
                legend(fontsize=12)
            

            return density, new_wavelengths[index_lr[0]:index_lr[1]], new_spectrum[index_lr[0]:index_lr[1]], params
        
        else:
            figure()
            title(r'$H_{\alpha}$ line fit',fontsize=12)
            plot(new_wavelengths[index_lr[0]:index_lr[1]],new_spectrum[index_lr[0]:index_lr[1]],'-',color='k',label='Signal')
                             
            for binder in [[voigt_profile_binder,'Voigt','b'],
                           [gaussian_profile_binder,'Gaussian','r'],
                           [lorentzian_profile_binder,'Lorentzian','g']]:
            
                current_profile_binder = binder[0]
                fit = binder[1]
                color1=binder[2]
                
            
                params, pcov = binder_fit(new_wavelengths[index_lr[0]:index_lr[1]], new_spectrum[index_lr[0]:index_lr[1]],
                                         current_profile_binder,
                                         initial_guess_x0 = line_ritz, initial_guess_A = new_spectrum[center_wavelength_peak_index])
                
                
                xx = np.arange(new_wavelengths[index_lr[0]],new_wavelengths[index_lr[1]],0.001)
                fwhm, fwhm_x_v, fwhm_y_v = fit_fwhm(xx,params,current_profile_binder)
                
                
                #density = 10e17*pow(fwhm/1.31,1/0.64)
                density = (fwhm/(2.8*10**-17))**(1/0.72)*10**-6
                
                
                fill_between(xx,current_profile_binder(xx,params[0],params[1],params[2],params[3]),color=color1,alpha=0.2,label=fit+' fit \n'+ '$n_{plasma} =$' + "%1.2e" % density +' $cm^{-3}$\n')
                plot(fwhm_x_v,fwhm_y_v,ls = ':', color = 'k', marker = '|',markersize = 10)
                ax = gca()
                
            
            ax.xaxis.set_major_formatter(ticker.FormatStrFormatter('%0.1f'))
            legend(fontsize=12)
                    
                    
                

            return density, new_wavelengths[index_lr[0]:index_lr[1]], new_spectrum[index_lr[0]:index_lr[1]], params

density_copper(list_of_experiments[4].mean_signal)

In [ ]:
sample_elements = [['Cu',1]]
sample = digital_twin(sample_elements)

print(sample)

temperature =.80*T_ref
wl,spec,label, n_ion, specs = sample.spectrum_NIST( lower_limit = ll, upper_limit = ul, electron_temperature=temperature,
                                                   max_ion_state=3,d_lambda=0.01, Map=False)
subplots()
plot(wl[0],specs[0],label='$T_p = 0.8 ev$')
print(n_ion)

temperature =.90*T_ref
wl,spec,label, n_ion, specs = sample.spectrum_NIST( lower_limit = ll, upper_limit = ul, electron_temperature=temperature,
                                                   max_ion_state=3,d_lambda=0.01, Map=False)
plot(wl[0],specs[0],label='$T_p = 0.9 ev$')
print(n_ion)

temperature =1.*T_ref
wl,spec,label, n_ion, specs = sample.spectrum_NIST( lower_limit = ll, upper_limit = ul, electron_temperature=temperature,
                                                   max_ion_state=3,d_lambda=0.01, Map=False)

plot(wl[0],specs[0],label='$T_p = 1.0 ev$')
print(n_ion)

legend()
